In [1]:
import os 
import nibabel as nib
import numpy as np
import pandas as pd

# <span style="color:#FFCC80">In vivo Swiss data processing </span> </br>
Compare and analyze if using optimized parameters improves or deteriorates the final local fields </br>

Keywords: </br>
* sw = slicewise
* pp = pipeline

In [2]:
root_dir = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing"

In [14]:
# We have 4 subjects inside
subjects = ['hc1', 'hc2', 'sci1', 'sci2']
measurements = ['m1', 'm2', 'm3', 'm4']
bgfr_algos = ['opt_pdf', 'opt_resharp', 'opt_sharp']
rows = []


In [19]:
# We have 4 subjects inside
subjects = ['hc1', 'hc2'] #, 'sci1', 'sci2']
measurements = ['m1', 'm2', 'm3', 'm4']
bgfr_algos = ['opt_lbv', 'opt_pdf', 'opt_resharp', 'opt_sharp']#,'opt_vsharp_8_to_1','def_iHarperella']
lf_rows = []

In [20]:
# Now lets collect rows for LF!

for subj in subjects:
    for meas in measurements:
        for algo in bgfr_algos:
            
            # Firsst, define the path pointers
            subj_meas_path = os.path.join(root_dir, subj, meas)
            lf_path = os.path.join(subj_meas_path, 'local_field', algo,'Sepia_localfield.nii.gz')
            gm_msk_path = os.path.join(subj_meas_path, f'{subj}_{meas}_gm_mks.nii.gz')
            wm_msk_path = os.path.join(subj_meas_path, f'{subj}_{meas}_wm_msk.nii.gz')

            # Load data
            lf_img = nib.load(lf_path)
            lf_data = lf_img.get_fdata()

            gm_mask = nib.load(gm_msk_path).get_fdata()
            total_vox_gm = np.sum(gm_mask==1) 
            wm_mask = nib.load(wm_msk_path).get_fdata()
            total_vox_wm = np.sum(wm_mask==1)

            # Compute metrics for GM and WM
            gm_mean = np.mean(lf_data[gm_mask==1])
            gm_std = np.std(lf_data[gm_mask==1])

            wm_mean = np.mean(lf_data[wm_mask==1])
            wm_std = np.std(lf_data[wm_mask==1])

            # Compute how many voxels are in the mask
            gm_nonzero_vox = np.sum(lf_data[gm_mask==1] != 0)
            wm_nonzero_vox = np.sum(lf_data[wm_mask==1] != 0)

            # Compute the contrast metric
            # We want to maximize the contrast between GM and WM
            # WM mean should be negative, if it is positive the contrast will be lower this way
            contrast = np.abs(gm_mean - wm_mean)
            normalizer_denominator = np.sqrt(gm_std**2 + wm_std**2)
            raw_metric = contrast / normalizer_denominator if normalizer_denominator != 0 else 0 # Just in case that the std is 0 - to avoid division by zero

            # Now we penalize if the algo eroded the mask
            gm_penality = gm_nonzero_vox / total_vox_gm 
            wm_penality = wm_nonzero_vox / total_vox_wm

            # Final metric:
            final_metric = raw_metric * gm_penality * wm_penality

            # Now collect row and add to data frame
            lf_rows.append({
                'subject': subj,
                'measurement': meas,
                'algo': algo,
                'mean_gm': gm_mean,
                'std_gm': gm_std,
                'total_vox_gm': total_vox_gm,
                'nonzero_vox_gm': gm_nonzero_vox,
                'mean_wm': wm_mean,
                'std_wm': wm_std,
                'total_vox_wm': total_vox_wm,
                'nonzero_vox_wm': wm_nonzero_vox,
                'contrast factor': contrast,
                'gm_penality': gm_penality,
                'wm_penality': wm_penality,
                'raw_metric': raw_metric,
                'final_metric': final_metric
            })


# Create the data frame
lf_df = pd.DataFrame(lf_rows)

# Save
lf_df.to_csv(os.path.join(root_dir, 'bgfr_metrics_complete_dubs_and_meass.csv'), index=False)
print(lf_df)

   subject measurement         algo   mean_gm    std_gm  total_vox_gm  \
0      hc1          m1      opt_lbv  0.530615  0.589751           645   
1      hc1          m1      opt_pdf  0.348916  1.140037           645   
2      hc1          m1  opt_resharp  0.217374  3.413937           645   
3      hc1          m1    opt_sharp  0.120387  2.674244           645   
4      hc1          m2      opt_lbv  0.468594  0.519468           647   
5      hc1          m2      opt_pdf  0.312296  0.754002           647   
6      hc1          m2  opt_resharp  0.346267  2.020135           647   
7      hc1          m2    opt_sharp  0.026899  1.389143           647   
8      hc1          m3      opt_lbv  0.371851  0.547411           623   
9      hc1          m3      opt_pdf  0.284571  1.040017           623   
10     hc1          m3  opt_resharp  0.382815  3.171093           623   
11     hc1          m3    opt_sharp  0.081665  1.807737           623   
12     hc1          m4      opt_lbv  0.293392  0.52